In [1]:
import sys
import matplotlib
import matplotlib.colors

sys.path.append("..")
import jabble.model
import jabble.dataset
import jabble.loss
import jabble.physics 
import astropy.units as u

import h5py 
import matplotlib.pyplot as plt
import scipy.optimize

# from jaxopt import GaussNewton
import jax.numpy as jnp
import jax
import numpy as np
# from mpl_axes_aligner import align

import os
import datetime
import jabble.physics

import jax
import jax.config

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_disable_jit", False)

/ext3/miniconda3/envs/wobbleenv/lib/python3.11/site-packages/jax/_src/api_util.py:174: SyntaxWarning: Jitted function has static_argnums=(3, 4, 5), but only accepts 5 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "



<h1>02 - Fitting RVs to HARPS e2ds Spectra</h1>
In this notebook, I am will show you how to load in data and metadata to jabble.data objects. Then fit models to each component of the data. Due to some edge errors with normalizing near the edge of the orders in the HARPS spectrograph, we fit a longer wavelength component normalization model.


Create output directory here as the current date.

In [2]:
today = datetime.date.today()
out_dir = os.path.join('/scratch/mdd423/wobble_jax/','out',today.strftime("%y-%m-%d"))
os.makedirs(out_dir,exist_ok=True)

In [3]:
out_dir

'/scratch/mdd423/wobble_jax/out/25-05-12'

In [4]:
file_b = h5py.File("../data/barnards_e2ds.hdf5", "r")
file_p = h5py.File("../data/51peg_e2ds.hdf5"   , "r")

In [5]:
file_b['dates']

<HDF5 dataset "dates": shape (306,), type "<f8">

In [6]:
jax.print_environment_info()

jax:    0.4.23
jaxlib: 0.4.23
numpy:  1.24.3
python: 3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:43:09) [GCC 12.3.0]
jax.devices (1 total, 1 local): [CpuDevice(id=0)]
process_count: 1



<h2>Jabble.Dataset</h2>
The 4 fields needed to create a jabble.data object are: xs, ys, yivar, and mask. xs is the input to whats being modeled, this should be a list of 1 dimensional jax.numpy arrays, which may have different shapes (lengths). These correspond to the log wavelength values being evaluated. The logarithm of wavelength means that the redshift from relative radial velocity change becomes an additive property.

$$x_{obs} = \log(\lambda_{obs})$$
$$\lambda_{obs} = \sqrt{\frac{1 + RV/c}{1- RV/c}}\lambda_{emit}$$
$$x_{obs} = \log(\sqrt{\frac{1 + RV/c}{1- RV/c}}\lambda_{emit})$$
$$x_{obs} = \log(\sqrt{\frac{1 + RV/c}{1- RV/c}}) + \log(\lambda_{emit})$$
$$x_{obs} = \delta x + x_{emit}$$

where $x_{obs}$ is the observed log wavelength, $RV$ is the radial velocity, $c$ is the speed of light, and $x_{emit}$ is the emitted log wavelength. ys is the measured log flux at that log wavelength, xs. This value is assumed to be in photon per wavelength bin. Thus, multiplicative properties like absorption from tellurics become additive in the logarithm. 

$$y = \log(f)$$
$$y = \log(f_s f_t) $$
$$y = \log(f_s) + \log(f_t) $$
$$y = y_s + y_t$$

yivar stands for y inverse variance, or y information. This value is produced by the instrumentation team at these detectors using some assumption about instrument readout and dark current properties along with poisson photon noise. Thus the error on the flux is not being modeled here and is assumed as correct from the pipeline. These errors are given as the standard deviation of the flux, so we need to transform error of f into the error in logarithm of flux.

$$\sigma_y = \sigma_f \frac{dy}{df}$$
$$\sigma_y = \frac{\sigma_f}{f}$$

In [7]:
def fit_continuum(x, y, ivars, order=6, nsigma=[0.3,3.0], maxniter=50):
    """Fit the continuum using sigma clipping

    Args:
        x: The wavelengths
        y: The log-fluxes
        order: The polynomial order to use
        nsigma: The sigma clipping threshold: tuple (low, high)
        maxniter: The maximum number of iterations to do

    Returns:
        The value of the continuum at the wavelengths in x
        Author: Megan Bedell

    """
    A = np.vander(x - np.nanmean(x), order+1)
    m = np.ones(len(x), dtype=bool)
    for i in range(maxniter):
        m[ivars == 0] = 0  # mask out the bad pixels
        w, _, _, _ = np.linalg.lstsq(np.dot(A[m].T, A[m]), np.dot(A[m].T, y[m]))
        mu = np.dot(A, w)
        resid = y - mu
        sigma = np.sqrt(np.nanmedian(resid**2))
        #m_new = np.abs(resid) < nsigma*sigma
        m_new = (resid > -nsigma[0]*sigma) & (resid < nsigma[1]*sigma)
        if m.sum() == m_new.sum():
            m = m_new
            break
        # print(m.shape,m_new.shape)
        m = m_new
    return mu

In [8]:
def get_dataset(file,orders,device):
    ys = []
    xs = []
    yivar = []
    mask = []

    init_rvs = np.array(file["bervs"])
    times_t = np.array([x for x in file["dates"]])
    full_init_rvs = []
    # print(file["bervs"].shape)
    airmass = []
    order_out = []
    times = []
    star_ids = []

    for iii in orders:
        for jjj in range(file["data"].shape[1]):
            contin = fit_continuum(np.array(file["xs"][iii,jjj,:]),np.array(file["data"][iii,jjj,:]),np.array(file["ivars"][iii,jjj,:]),\
                                   order=6, nsigma=[0.3,3.0], maxniter=50)
            
            ys.append(jnp.array(file["data"][iii,jjj,:]) - contin)
            xs.append(jnp.array(file["xs"][iii,jjj,:]))
            yivar.append(jnp.array(file["ivars"][iii,jjj,:]))
            mask.append(jnp.zeros(file["data"][iii,jjj,:].shape,dtype=bool))

            # init_shifts.append(
            airmass.append(file["airms"][jjj])
            order_out.append(iii)
            times.append(file["dates"][jjj])
            star_ids.append(str(file))

            full_init_rvs.append(file["bervs"][jjj])
    
    full_init_rvs = jnp.array(full_init_rvs)
    airmass = jnp.array(airmass)

    dataset = jabble.dataset.Data.from_lists(xs,ys,yivar,mask)

    dataset.metakeys['times'] = times_t
    dataset.metadata['times'] = np.array(times)
    dataset.metadata['orders'] = np.array(order_out)
    dataset.metadata['star_ids'] = np.array(star_ids)
    dataset.metadata['init_rvs'] = np.array(full_init_rvs)

    dataset.metadata['airmass'] = airmass
    dataset.metadata['bervs'] = full_init_rvs
    dataset.to_device(device)
    
    # init_shifts = jax.device_put(init_shifts,device)
    # airmass = jax.device_put(airmass,device)
    
    return dataset, init_rvs, airmass, full_init_rvs, times_t

<h2>Jabble.Model</h2>
Now, we will define the components of the model. These stars will have 3 components: stellar, telluric, pseudo-normalization. 

$$y = y_\mathrm{s} + y_\mathrm{t} + y_\mathrm{N}$$

The stellar component will be be high resolution (twice R of the detector) in wavelength and static in flux across all times or epochs of data. However, a nonlinear comes from the redshift radial velocity being fit inside the linear model log flux model. Because our dataset will consist of different orders at the same time, and the same order at different time, and we are assuming the wavelenght solution to be true. Thus the redshift should be the same at different orders at the same time, so $i_*$ is the index of the epoch while $i$ is the data index, which is different for all orders and times or can be thought of simply as the data index.

$$y_s = f(x + \delta x_{i_*}|\theta_\mathrm{s})$$

The tellurics model is also high resolution in wavelength and static across all times, but the airmass is used to stretch the model at each model at each time. This parameter is not fit, however is can be if the user chooses to do so!

$$y_t = a_i f(x|\theta_\mathrm{t})$$

And finally the pseudo-normalization model is low resolution in wavelength about one parameter per 50 Angstroms and is different across all times because we expect the detector state to vary greatly with time.

$$y_\mathrm{N} = f(x|\theta_{\mathrm{N},i})$$

The function we choose to represent the flux of these models is the cardinal spline mixture, who are evenly spaced in wavelength. 

$$f(x|\theta,p) = \sum_i \theta_i k(x|p,i)$$

where $k$ is the cardinal basis/kernel function. This basis function is a piecewise polynomial positioned on an evenly spaced grid such that the total function will be continuous in $p-1$ derivatives.

In [9]:
import jabble.quickplay
def get_model(dataset,resolution,p_val,vel_padding,init_rvs,rest_vels,airmass,pts_per_wavelength,norm_p_val):
         
    dx = jabble.physics.delta_x(2 * resolution)
    x_grid = jnp.arange(np.min(np.concatenate(dataset.xs)), np.max(np.concatenate(dataset.xs)), step=dx, dtype="float64")
    
    model_grid = jabble.model.create_x_grid(
        x_grid, vel_padding.to(u.m/u.s).value, 2 * resolution
    )  
    # model = jabble.quickplay.get_wobble_model(init_rvs, airmass, model_grid, p_val,rest_vels) #+ jabble.quickplay.get_normalization_model(dataset,norm_p_val,pts_per_wavelength)

    # model = jabble.quickplay.PseudoNormalModel(init_shifts, init_airmass, model_grid, p_val, dataset, norm_p_val, pts_per_wavelength)
    init_shifts = jabble.physics.shifts(init_rvs)
    rest_shifts = jabble.physics.shifts(rest_vels)
    model = jabble.model.CompositeModel(
        [
            jabble.model.ShiftingModel(init_shifts,which_key='times'),#ShiftingModel(init_shifts),#
            jabble.model.CardinalSplineMixture(model_grid, p_val),
        ]
    ) + jabble.model.CompositeModel(
        [
            jabble.model.ShiftingModel(rest_shifts,which_key='times'),
            jabble.model.CardinalSplineMixture(model_grid, p_val),
            jabble.model.StretchingModel(airmass),
        ]
    ) #+ jabble.model.get_normalization_model(dataset,norm_p_val,pts_per_wavelength)

    # model.to_device(device)

    # model.fit(2)
    # print(type(model.get_parameters()),model.get_parameters().devices())
    return model

<h2>Training Cycle</h2>
Now we define the fitting process of the parameters of all of these models with respect to the total set of data. This is require some clever initialization, so that we don't end up too far from the solution. First, we fit the parameters of the tellurics, stellar, normalization model. Regularizing the stellar and tellurics components, so that they are close 0. Then we fit the RV alone. Then we optimize everything together with regularization on the stellar and tellurics component.

In [10]:
def train_cycle(model, dataset, loss, lmb, device_store, device_op, batch_size):
    options = {"maxiter": 4096}
    
    # Fit Normalization Template
    # model.fix()
    # model.fit(2, 1)
    # # model.fit(0,1)
    # # model.fit(1,1)
    # # model.display()
    reg_s = lmb*jabble.loss.L2Reg([0,1])
    reg_t = lmb*jabble.loss.L2Reg([1,1])
    
    # res1 = model.optimize(loss, dataset, device_store, device_op, batch_size, options=options)#model.optimize(loss, dataset)
    # print(res1)

    # Fit Stellar & Telluric Template
    model.fix()
    model.fit(0,1)
    model.fit(1,1)
    model.display()
    
    res1 = model.optimize(loss + reg_s + reg_t, dataset, device_store, device_op, batch_size, options=options)#model.optimize(loss, dataset)
    print(res1)
    
    # Fit RV
    model.fix()
    model.fit(0,0)
    res1 = model.optimize(loss, dataset, device_store, device_op, batch_size, options=options)
    print(res1)

    # RV Parabola Fit
    # model.fix()
    # shift_search = jabble.physics.shifts(np.linspace(-50, 50, 100))
    # model[0][0].parabola_fit(shift_search, loss, model, dataset, device_op, device_store)
    # model.to_device(device_op)
    # print(type(model_p[0][0].p))

    # Fit Everything
    model.fix()
    model.fit(0,0)
    model.fit(0,1)
    model.fit(1,1)

    res1 = model.optimize(loss + reg_s + reg_t, dataset, device_store, device_op, batch_size, options=options)#model.optimize(loss, dataset)
    print(res1)

    return model

Here we define the 'loss' to be the sum of the chi squared of each flux. This is typical choice when using jabble, and why we must define the inverse variance of y, $I_{y}$ in the data object.

$$\mathcal{L}_{\chi^2} = \sum I_y (ys - y(xs|\delta x_{i_*},\theta_\mathrm{s},\theta_\mathrm{t},\theta_{\mathrm{N},i}) )^2$$

$xs, ys, I_{y}$ are from the data and are then summed over all unmasked elements.

$$\mathcal{L}_{\mathrm{reg}} = \sum \theta_\mathrm{s}^2 + \theta_\mathrm{t}^2$$

$$\mathcal{L}_{total} = \mathcal{L}_{\chi^2} + \lambda \mathcal{L}_{\mathrm{reg}}$$

In [11]:
cpus = jax.devices("cpu")
# gpus = jax.devices("gpu")
loss = jabble.loss.ChiSquare()

Here we define the parameters of the instrument that are needed to build our model. The instrument is HARPS, so the resolution is 115,000. The $p$ value for the CSM model on the stellar and telluric components is set to 2. This means just the first derivative of the model will be continuous. And we will add a wavelength padding to either side of the log wavelength grid out to $100$ km/s in log wavelength. And the lambda for the regularization term is defined here as $\lambda = 100$.

In [12]:
resolution = 115_000
p_val = 3
vel_padding = 100 * u.km / u.s


pts_per_wavelengths = [1/25.0]
norm_p_vals = [3]
lmbs = [1000.0]

star_name_b, star_name_p = 'barnards','peg51'

We will not fit the entire spectrum in one pass but we will fit chunks of multiple orders with one RV per epoch.

In [13]:
orders_s = np.arange(0,72,dtype=int)[:,None].reshape(-1,2)
print(orders_s)
# orders_s = [[34]]

[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]
 [20 21]
 [22 23]
 [24 25]
 [26 27]
 [28 29]
 [30 31]
 [32 33]
 [34 35]
 [36 37]
 [38 39]
 [40 41]
 [42 43]
 [44 45]
 [46 47]
 [48 49]
 [50 51]
 [52 53]
 [54 55]
 [56 57]
 [58 59]
 [60 61]
 [62 63]
 [64 65]
 [66 67]
 [68 69]
 [70 71]]


Also decide what device you want to store extra data on, and which device to put the model and do the fitting on.

In [14]:
file_b.filename

'../data/barnards_e2ds.hdf5'

In [15]:
# device_store = cpus[0]
# device_op = gpus[0]
# batch_size = 5000

# for orders in orders_s:
#     for star_name,file in zip([star_name_b,star_name_p],[file_b,file_p]):
#         os.makedirs(os.path.join(out_dir,star_name),exist_ok=True)
#         for lmb in lmbs:
#             for norm_p_val in norm_p_vals:
#                 for pts_per_wavelength in pts_per_wavelengths:
#                     data_name = os.path.join(out_dir,star_name,star_name + '_data_o{}_pre_norm.pkl'.format(orders))
#                     model_name = os.path.join(out_dir,star_name,star_name + '_model_o{}_pre_norm.pkl'.format(orders))
#                     dataset, init_rvs, airmass, full_init_rvs, times_t = get_dataset(file,orders,device_op)

#                     # print(init_rvs.shape,full_init_rvs.shape)
#                     datablock, metablock, meta_keys = dataset.blockify(device_store,return_keys=True)
#                     # print(datablock,metablock)
#                     # for datarow in datablock:
#                     #     print(datarow)
#                     # break

#                     rest_rvs = jnp.zeros(init_rvs.shape)
#                     model = get_model(dataset,resolution,p_val,vel_padding,-init_rvs,rest_rvs,airmass,pts_per_wavelength,norm_p_val)
                    
#                     # break
#                     model.metadata["dataset"] = file.filename
#                     for key in dataset.metadata.keys():
#                         model.metadata[key] = dataset.metadata[key]
                    
#                     model.to_device(device_op)
#                     reg_s = lmb*jabble.loss.L2Reg([0,1])
#                     reg_t = lmb*jabble.loss.L2Reg([1,1])
#                     model = train_cycle(model, dataset, loss, lmb, device_store, device_op, batch_size)
#                     # model.display()

#                     model.fix()
#                     model.fit(0,1)
#                     model.fit(1,1)
#                     reg_s.ready_indices(model)
#                     reg_t.ready_indices(model)
#                     jabble.quickplay.save(model,model_name,data_name,data=dataset,shifts=model[0][0].p,device=device_op,loss=loss+reg_s+reg_t)
#                     # jabble.quickplay.save(model,model_name,mode="pkl",data=dataset,device=device_op)

#                     # with h5py.File(model_name + '.hdf','r') as file:
#                     # print(model_name + '.hdf',file.keys())
#                     # model = jabble.quickplay.load(model_name + '.hdf','hdf')
#                         # for key in file.keys():
#                         #     cls = eval('jabble.model.' + key)
#                         #     model = cls.load_hdf(cls,file)
#                         #     model.display()
#                         #     print(model.results)
#                         # print(file['AdditiveModel']['[0]CompositeModel']['[0][1]CardinalSplineMixture']['p_val'])
#                     # break
#                     # jabble.quickplay.load()
#                     # jabble.model.save(model_name,model)

In [16]:
def make_deriv_plots(dataset,model,plt_epoch,device,plt_name):
    l_width = np.arange(-0.5,0.5,0.2) + 11.5
    
    fig, axes = plt.subplots(3,figsize=(5,5),sharex='col',\
                             facecolor=(1, 1, 1),height_ratios=[1,1,1],dpi=200)
    data, meta, keys = dataset.blockify(device,return_keys=True)

    datarow = jabble.loss.dict_ele(data,plt_epoch,device)
    metarow = jabble.loss.dict_ele(meta,plt_epoch,device)
    
    lrange = np.floor(np.exp(datarow['xs']).mean()) + l_width

    xplot = np.linspace(np.log(lrange.min()),np.log(lrange.max()),\
                        dataset.xs[plt_epoch].shape[0]*10)

    model.fix()
    yplot_norm_tot  = model([],xplot,metarow)
    yplot_norm_stel = model[0]([],xplot,metarow)
    yplot_norm_tell = model[1]([],xplot,metarow)
    yhat = model([],dataset.xs[plt_epoch][:],metarow)
    
    velocity = jabble.physics.velocities(model[0][0].p[plt_epoch])
    
    # Data
    axes[0].errorbar(datarow["xs"][:],datarow["ys"][:],yerr=1/np.sqrt(datarow["yivar"][:]),fmt='ok',zorder=3,alpha=0.5,ms=1)

    # Stellar Model        
    axes[0].plot(xplot,yplot_norm_stel,'-r',linewidth=1.2,zorder=2,alpha=0.5,ms=6)
    # Telluric Model
    axes[0].plot(xplot,yplot_norm_tell,'-b',linewidth=1.2,zorder=1,alpha=0.5,ms=6)
    axes[0].set_yticks([])
    # axes[0].set_ylim(-3,1)
    axes[0].set_ylabel('f')
    
    model.fix()
    model.fit(0,0)
    deriv_model = jax.jacfwd(model,argnums=0)(model.get_parameters(),datarow['xs'],metarow)
    deriv_model_plot = jax.jacfwd(model,argnums=0)(model.get_parameters(),xplot,metarow)
    print(deriv_model.shape)
    # deriv_axes = axes[0].twinx()
    axes[1].plot(xplot,deriv_model_plot[:,plt_epoch],'y',alpha=0.5,ms=6)
    axes[1].plot(xplot,np.zeros(xplot.shape[0]),'-k',alpha=0.3,linewidth=1,zorder=-1)
    axes[1].set_xlim(np.log(lrange.min()),np.log(lrange.max()))
    axes[1].set_yticks([])
    axes[1].set_ylabel(' $\\frac{\\partial f}{\\partial \\delta x}$')

    # info_axes = axes[0].twinx()
    axes[2].step(datarow['xs'],deriv_model[:,plt_epoch]*datarow['yivar']*deriv_model[:,plt_epoch],where='mid',alpha=0.5,ms=4,c='g')
    axes[2].plot(xplot,np.zeros(xplot.shape[0]),'-k',alpha=0.3,linewidth=1,zorder=-1)

    axes[2].set_xlim(np.log(lrange.min()),np.log(lrange.max()))
    axes[2].set_yticks([])
    axes[2].set_ylabel(' $\\frac{\\partial f}{\\partial \\delta x} I_{y} \\frac{\\partial f}{\\partial \\delta x}$')
    # deriv_axes.set_ylim(-1e4,1e4)

    axes[0].set_xlim(xplot.min(),xplot.max())
    axes[0].set_xticks(np.log(lrange))
    axes[1].set_xticklabels(['{:0.1f}'.format(x) for x in lrange])
    axes[0].set_xlim(np.log(lrange.min()),np.log(lrange.max()))
        
    fig.text(0.5, 0.00, 'Wavelength $[\AA]$', ha='center')
    # fig.text(0.08, 0.5, 'Normalized Flux', va='center', rotation='vertical')
    if plt_name is not None:
        plt.savefig(os.path.join(out_dir, plt_name),bbox_inches='tight')
    plt.show()

In [17]:
# make_deriv_plots(all_data[0],all_models[0],40,cpus[0],'info.png')

In [18]:
# def get_savable_attr(model):
#     for key in model.__dict__:
#         # print(key)
#         if key[0] != "_":
#             # print(key)
#             if '__len__' in dir(model.__dict__[key]):
#                 if len(model.__dict__[key]) != 0:
#                     print(key,model.__dict__[key])
#                     if key == "models":
#                        for submodel in model.models:
#                             get_savable_attr(submodel)
# get_savable_attr(model)

In [19]:
import glob
dir = glob.glob('/scratch/mdd423/wobble_jax/out/25-05-08/barnards/*.pkl')
print(len(dir))

72


In [20]:
# import inspect
# lines = inspect.getsource(jabble.quickplay.save)
# print(lines)

In [21]:
# def recurse_hdf(file):
#     print(file.keys())
#     for key in file.keys():
#         # print(key)
#         if isinstance(file[key], h5py.Group):
            
#             recurse_hdf(file[key])
#         if key == "results":
#             print("yes")

# def load_hdf(filename):
#     with h5py.File(filename,'r') as file: 
#         print(filename)
#         for key in file.keys():
#             print(file,key,type(file[key]))
#             if isinstance(file[key], h5py.Group):
#                 print(key)
#                 cls = eval(key)
#                 model = cls.load_hdf(cls,file[key])
#                 # recurse_hdf(group[key])
#     return model
# #model_name + '.hdf'
# with h5py.File(dir[0],'r') as file: 
#     # recurse_hdf(dir[0])
#     print(file.keys())
#     # for key in file.keys():
#     #     print(key)
#     #     if key in dir(jabble.model):
#     #         print("yes")
# # load_hdf(dir[0])

In [22]:
out_dir

'/scratch/mdd423/wobble_jax/out/25-05-12'

In [23]:
summary_path = "/scratch/mdd423/wobble_jax/out/25-05-08/peg51"
dir_files = glob.glob(os.path.join(summary_path,'*_RVS.hdf'))

In [24]:
print([*zip(np.arange(0,4).reshape(2,2),np.ones(2))])

[(array([0, 1]), 1.0), (array([2, 3]), 1.0)]


In [26]:
rv_array, all_models, all_rv_array, all_data = jabble.quickplay.load_model_dir(summary_path,dir_files,cpus[0],force_run=True)

In [ ]:
# all_rv_array = all_rv_array[np.argsort(all_rv_array['min_order'])]

In [ ]:
all_rv_array['Loss_Avg'].shape,all_rv_array['RV_all'].shape,all_rv_array['Time_all'].shape

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
def rv_all_order_plot(times,rv_e,err_e,time_comb,rv_comb,err_comb,targ_time,targ_vel,targ_err,bervs):
    fig, ax = plt.subplots(
        1,
        figsize=(6, 4),
        facecolor=(1, 1, 1),
        dpi=300,
        sharey=True
    )
    temp_vel  = targ_vel - (targ_vel)
    temp_vel -= temp_vel.mean()
    ax.errorbar(targ_time,temp_vel,targ_err,fmt='.g',zorder=3,alpha=0.30,ms=2,label='HARPS RV')

    targ_ind = np.argsort(targ_time)
    comb_indi = np.argsort(np.argsort(time_comb))
    temp_vel  = rv_comb - targ_vel[targ_ind][comb_indi]
    temp_vel -= temp_vel.mean()
    ax.errorbar(time_comb,temp_vel,err_comb,fmt='.r',zorder=5,alpha=0.50,ms=2,label='Order Jabble Combined RV')

    # temp_vel = rv_e.mean(axis=0) + bervs
    # temp_vel -= temp_vel.mean()
    # ax.errorbar(times % period,temp_vel,0.0,fmt='.b',zorder=2,alpha=0.60,ms=2,label='Avg RV')

    for i in range(rv_e.shape[0]):
        targ_ind = np.argsort(targ_time)
        comb_indi = np.argsort(np.argsort(times[i,:]))
        temp_vel  = rv_e[i,:] - targ_vel[targ_ind][comb_indi]
        temp_vel -= temp_vel.mean()
        ax.errorbar(times[i,:],temp_vel,yerr=err_e[i,:],fmt='.k',zorder=2,alpha=0.05,ms=2,label='Order Jabble RV')

    # fig.legend()
    ax.set_ylim(-500, 500)
    # fig.legend()
    ax.set_title('Barnard\'s Star Relative Radial Velocities')
    ax.set_ylabel("RV [$m/s$]")
    ax.set_xlabel( "MJD")
    plt.savefig(os.path.join(out_dir, "barn_rvs_time.png"))

    # ax.set_xlim(2.4564e6,2.45644e6)
    # ax.set_ylim(-20e3,-10e3)
    # plt.savefig(os.path.join(out_dir, "02-barns_all_order_nobervs_epoch.png"))
    plt.show()

def rv_all_order_harps_plot(times,rv_e,err_e,times_comb,rv_comb,err_comb,targ_time,targ_vel,\
                            targ_err,bervs,loss_array,rv_difference_array,star_name):
    
    epoches_span = np.arange(0,len(times_comb),dtype=int)

    # RV Err Comparison
    
    fig, ax = plt.subplots(
        1,
        figsize=(10, 4),
        facecolor=(1, 1, 1),
        dpi=300,
        sharey=True
    )
    targ_ind = np.argsort(targ_time)
    comb_indi = np.argsort(np.argsort(times_comb))

    bervs_temp = -bervs[targ_ind][comb_indi] + bervs.mean()
    norm_vel   = bervs_temp #(targ_vel[targ_ind][comb_indi] - targ_vel[targ_ind][comb_indi].mean())
    # print(np.sum(times_comb == targ_time[targ_ind][comb_indi]))

    # targ_norm  = (targ_vel[targ_ind][comb_indi] - targ_vel[targ_ind][comb_indi].mean()) - norm_vel
    targ_line = ax.plot(epoches_span,targ_err[targ_ind][comb_indi],'.g',zorder=3,alpha=0.5,ms=2,label='HARPS RV')

    
    # comb_norm = (rv_comb - rv_comb.mean()) - norm_vel
    comb_line = ax.plot(epoches_span,err_comb,'.r',zorder=1,alpha=0.3,ms=2,label='Order Jabble Combined RV')

    # ax.set_title('Barnard\'s Star Relative Radial Velocity Error')
    ax.set_ylabel("RV Error [$m/s$]")
    ax.set_xlabel( "Epochs")
    # ax.set_xlim(2.4564e6,2.45644e6)
    # ax.set_ylim(-1e2,1e2)
    plt.savefig(os.path.join(out_dir, "{}_rvs_err_epoch.png".format(star_name)),bbox_inches='tight')
    plt.show()
    
    # RV Comparison
    
    fig, ax = plt.subplots(
        1,
        figsize=(10, 4),
        facecolor=(1, 1, 1),
        dpi=300,
        sharey=True
    )
    targ_ind = np.argsort(targ_time)
    comb_indi = np.argsort(np.argsort(times_comb))

    bervs_temp = -bervs[targ_ind][comb_indi] + bervs.mean()
    norm_vel   = bervs_temp #(targ_vel[targ_ind][comb_indi] - targ_vel[targ_ind][comb_indi].mean())
    # print(np.sum(times_comb == targ_time[targ_ind][comb_indi]))

    targ_norm  = (targ_vel[targ_ind][comb_indi] - targ_vel[targ_ind][comb_indi].mean()) - norm_vel
    targ_line = ax.errorbar(epoches_span,targ_norm,targ_err[targ_ind][comb_indi],fmt='.g',zorder=3,alpha=0.5,ms=2,label='HARPS RV')

    
    comb_norm = (rv_comb - rv_comb.mean()) - norm_vel
    comb_line = ax.errorbar(epoches_span,comb_norm,err_comb,fmt='.r',zorder=2,alpha=0.3,ms=2,label='Order Jabble Combined RV')

    # temp_vel = (rv_e.mean(axis=0) - rv_e.mean()) #+ bervs_temp[targ_ind][comb_indi]
    # avg_line = ax.errorbar(epoches_span,temp_vel,0.0,fmt='.b',zorder=2,alpha=0.0,ms=2,label='Avg RV')

    for i in range(rv_e.shape[0]):
        e_ind = np.argsort(times[i,:])
        indiv_norm = (rv_e[i,:][e_ind][comb_indi] - rv_e[i,:].mean()) - norm_vel
        # times[i,:][e_ind][comb_indi]
        err_line = ax.errorbar(epoches_span,indiv_norm,yerr=err_e[i,:][e_ind][comb_indi],fmt='.k',zorder=1,alpha=0.03,ms=2,label='Order Jabble RV')

    # ax.set_ylim(-100, 100)
    ax.legend(handles=[targ_line,comb_line,err_line],loc="upper right")

    # ax.set_title('Barnard\'s Star Relative Radial Velocities')
    ax.set_ylabel("RV [$m/s$]")
    ax.set_xlabel( "Epochs")
    # ax.set_xlim(2.456e6 + 451,2.456e6+452)
    ax.set_ylim(-50,50)
    plt.savefig(os.path.join(out_dir, "{}_rvs_epoch.png".format(star_name)),bbox_inches='tight')
    plt.show()
    
    # FIGURE 2
    # fig, ax = plt.subplots(
    #     1,
    #     figsize=(10, 4),
    #     facecolor=(1, 1, 1),
    #     dpi=300,
    #     sharey=True
    # )

    # for i in range(rv_e.shape[0]):
    #     ax.errorbar(times[i,:],rv_e[i,:],yerr=err_e[i,:],fmt='.k',zorder=1,alpha=0.05,ms=2,label='Order Jabble RV')
    # ax.errorbar(times_comb,rv_comb,err_comb,fmt='.r',zorder=2,alpha=0.3,ms=2,label='Order Jabble Combined RV')

    # ax.set_ylim(-100, 100)
    # ax.set_title('Combination Check')
    # ax.set_ylabel("RV [$m/s$]")
    # ax.set_xlabel( "MJD")
    # ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%5.1f'))
    # plt.savefig(os.path.join(out_dir, "{}_rv_comb_check.png".format(star_name)),bbox_inches='tight')
    # plt.show()

    # RV DIFFERENCE PLOT
    fig, axes = plt.subplots(
        3,
        figsize=(10, 4),
        facecolor=(1, 1, 1),
        dpi=300,
        sharey=True
    )
    im1 = axes[0].imshow(rv_difference_array,interpolation ='nearest',vmin=-10,vmax=10)
    fig.colorbar(im1,ax=axes[0])
    axes[0].set_ylabel('$\Delta$ RV')
    
    im2 = axes[1].imshow(err_e,interpolation ='nearest',vmin=0,vmax=3)
    fig.colorbar(im2,ax=axes[1])
    axes[1].set_ylabel('$\sigma_{RV}$')
    
    im3 = axes[2].imshow(rv_difference_array/err_e,interpolation ='nearest',vmin=-3,vmax=3)
    print(np.sqrt(np.mean((rv_difference_array/err_e)**2)))
    fig.colorbar(im3,ax=axes[2])
    axes[2].set_ylabel('$\Delta$ RV /$\sigma_{RV}$')
    
    axes[0].set_xticks([])
    axes[1].set_xticks([])
    plt.xlabel('Epochs')
    plt.savefig(os.path.join(out_dir,'{}_drv.png'.format(star_name)),bbox_inches='tight')
    plt.show()

    plt.hist((rv_difference_array/err_e).flatten(),bins=30)
    plt.title('$\Delta$ RV /$\sigma_{RV}$')
    plt.show()
    
    # LOSS ARRAY PLOT
    plt.figure(figsize=(12,6),facecolor=(1, 1, 1),dpi=300)
    im = plt.imshow(loss_array,interpolation ='nearest',vmin=0,vmax=10000)

    xs,ys = np.where(np.isnan(loss_array))
    plt.scatter(ys,xs,s=10,facecolors='none', edgecolors='b')

    xs,ys = np.where(np.isinf(loss_array))
    plt.scatter(ys,xs,s=10,facecolors='none', edgecolors='r')
    plt.xlabel('Epochs')
    plt.ylabel('Chunks')
    plt.title('$\chi^2$')
    plt.colorbar(im,shrink=0.3)  
    plt.savefig(os.path.join(out_dir,'{}_loss.png'.format(star_name)),bbox_inches='tight')
    plt.show()

    # RV difference divided by RV error
    
    plt.show()


    # loss = np.array([model.results[-1][''] for model in all_models
    # loss_mean = np.mean(loss,axis=3).mean(axis=1)
    # print(loss_mean.shape)

    # fig, ax = plt.subfigures((2,2),figsize=(12,6),facecolor=(1, 1, 1),dpi=300,height_ratios=[1,4],width_ratios=[4,1],sharex='col',sharey='row')
    # im = ax[1,0].imshow(loss_mean,interpolation ='nearest',vmin=0,vmax=10)
    
    
    # ax[0,1].axis('off')
    # plt.xlabel('epoches')
    # plt.ylabel('chunks')
    # plt.title('$\chi^2$')
    # plt.colorbar(im,shrink=0.3)
    # plt.savefig(os.path.join(out_dir,'barn_obj.png'))
    # plt.show()

In [ ]:
# rv_all_order_plot(all_tim,all_rvs,all_err,\
#                   rv_array['Time_comb'],rv_array['RV_comb'],rv_array['RV_err_comb'],\
#                   np.array(file_b['dates']),np.array(file_b['pipeline_rvs']),np.array(file_b['pipeline_sigmas']),\
#                   np.array(file_b['bervs']))
star_name = "peg51"
rv_all_order_harps_plot(all_rv_array['Time_all'],all_rv_array['RV_all'],all_rv_array['RV_err_all'],\
                        rv_array['Time_comb'],rv_array['RV_comb'],rv_array['RV_err_comb'],\
                        np.array(file_p['dates']),np.array(file_p['pipeline_rvs']),np.array(file_p['pipeline_sigmas']),\
                        np.array(file_p['bervs']),all_rv_array['Loss_Avg'],all_rv_array['RV_difference'],star_name)

In [ ]:
all_rv_array.dtype,all_rv_array['RV_all'].shape,all_rv_array['RV_err_all'].shape,\
all_rv_array['Time_all'].shape,all_rv_array['Loss_Avg'].shape,all_rv_array['RV_difference'].shape,

In [ ]:
all_rv_array["RV_all"].shape

In [ ]:
from astropy.io import fits

In [ ]:
line_list = fits.open("~/wobble_jax/data/J_ApJ_795_23_table2.dat.fits")

In [ ]:
line_list[0].header

In [ ]:
# cmap = cm.get_cmap("Spectral")
cmap = matplotlib.colormaps["Spectral"]
# import cmastro

In [ ]:
def make_subplot(axes,model,dataset,plt_epoch,device,lrange,line_list = None):

    model.fix()
    data, meta, keys = dataset.blockify(device,return_keys=True)
    
    datarow = jabble.loss.dict_ele(data,plt_epoch,device)
    metarow = jabble.loss.dict_ele(meta,plt_epoch,device)
    # fig.suptitle("Order {}".format(keys["orders"][meta["orders"][plt_epoch]]))
    # axes[0,ii].title.set_text('Date: {}'.format(keys["times"][meta["times"][plt_epoch]]))

    # print(metarow['index'],plt_epoch)
    
    xplot = np.linspace(np.log(lrange.min()),np.log(lrange.max()),\
                        dataset.xs[plt_epoch].shape[0]*10)

    yplot_norm_tot  = model([],xplot,metarow)
    yplot_norm_stel = model[0]([],xplot,metarow)
    yplot_norm_tell = model[1]([],xplot,metarow)
    # yplot_norm      = model[2]([],xplot,metarow)
    # for epoch in np.where(indices):
    yhat = model([],dataset.xs[plt_epoch][:],metarow)
    axes[0].set_xlim(xplot.min(),xplot.max())

    velocity = jabble.physics.velocities(model[0][0].p[plt_epoch])
    
    # Data
    # print(datarow)
    axes[0].errorbar(datarow["xs"][:],datarow["ys"][:],yerr=1/np.sqrt(datarow["yivar"][:]),fmt='.k',zorder=2,alpha=0.1,ms=5)

    # Stellar Model        
    axes[0].plot(xplot,yplot_norm_stel,'-r',linewidth=1.2,zorder=10,alpha=0.7,ms=6)
    # Telluric Model
    axes[0].plot(xplot,yplot_norm_tell,'-b',linewidth=1.2,zorder=10,alpha=0.7,ms=6)
    # Total
    # axes[0].plot(xplot,yplot_norm_tot,'-m',linewidth=1.2,zorder=10,alpha=0.7,ms=6)
    # Norm
    # axes[0,ii].plot(xplot,yplot_norm,'-g',linewidth=1.2,zorder=10,alpha=0.7,ms=6)
    # Theory Model
    # theory_ax = axes[0,ii].twinx()
    # theory_ax.plot(dataset_theory.xs[0][:],dataset_theory.ys[0][:],'-y',linewidth=1.2,zorder=10,alpha=0.7,ms=6)
    # theory_ax.set_ylim(-5,5)
    # Line List
 
    # Residuals
    axes[1].step(dataset.xs[plt_epoch],dataset.ys[plt_epoch] - yhat,\
                             'k',where='mid',zorder=1,alpha=0.3,ms=3)

    axes[0].text(np.log(lrange.min()+0.1),0.3,"Order: ${}$".format(model.metadata["orders"][plt_epoch]))
    # axes[0].text(np.log(lrange.max()-0.1),0.3,"MJD: ${}$".format(model.metadata["times"][plt_epoch]))

    axes[0].set_ylim(-2.5,0.5)
    axes[1].set_ylim(-1,1)
    
    # axes[0].set_xticks([])
    axes[0].set_xticks(np.log(lrange))
    axes[1].set_xticks(np.log(lrange))

    axes[0].set_xticklabels(['' for x in lrange])
    axes[1].set_xticklabels(['{:0.1f}'.format(x) for x in lrange])
    
    axes[0].set_xlim(np.log(lrange.min()),np.log(lrange.max()))
    axes[1].set_xlim(np.log(lrange.min()),np.log(lrange.max()))

    return axes

def make_grid_plots(datasets,models,size_n,size_m,plt_epochs,device,plt_name,line_list=None):
    l_width = np.arange(-4,4,2)
    
    fig, axes = plt.subplots(2*size_n,size_m,figsize=(5*size_m,5*size_n),sharey='row',\
                             facecolor=(1, 1, 1),height_ratios=[4,1]*size_n,dpi=200)
    # plt.tight_layout()

    for ii, nn in enumerate(range(size_n)):
        for jj, mm in enumerate(range(size_m)):
            this_index = size_m*ii+jj
            
            lrange = np.floor(np.exp(datasets[this_index].xs[plt_epochs[this_index]]).mean()) + l_width
            # print(lrange)
            axes[2*ii:((2*ii)+2),jj] = make_subplot(axes[2*ii:((2*ii)+2),jj],models[this_index],datasets[this_index],0,device,lrange)
            if line_list is not None:
                plot_line_list(axes[2*ii:((2*ii)+2),jj],line_list,lrange)

    
    plt.subplots_adjust(wspace=0.15)
    # fig.text(0.5, 0.00, 'Wavelength $[\AA]$', ha='center')
    # fig.text(0.08, 0.5, 'Normalized Flux', va='center', rotation='vertical')
    if plt_name is not None:
        plt.savefig(os.path.join(out_dir, plt_name),bbox_inches='tight')
    plt.show()


def make_order_plot(dataset,model,lrange,plt_epoches,device,plt_name=None,line_list=None):
    # model = jabble.model.load(model_name)
    data_orders = np.unique(model.metadata["orders"])
    model.fix()
        
    fig, axes = plt.subplots(2,len(plt_epoches),figsize=(4*len(plt_epoches),4),sharex='col',sharey='row',\
                             facecolor=(1, 1, 1),height_ratios=[4,1],dpi=200)

    for ii, plt_epoch in enumerate(plt_epoches):
        axes[:,ii] = make_subplot(axes[:,ii],model,dataset,plt_epoch,device,lrange[ii,:])#(axes,model,dataset,plt_epoch,device,lrange)
        if line_list is not None:
            plot_line_list(axes[:,ii],model,line_list,lrange[ii,:],plt_epoch)
    
    # plt.x
    # plt.text(1, 1, 'Wavelength ($\AA$)', ha='center')
    # plt_name = os.path.join(out_dir, "02-spectra_{}_{}-{}.png".format(os.path.split(model_name)[-1],lmin,lmax))
    # plt.savefig(plt_name,dpi=200,bbox_inches='tight')
    # fig.suptitle('Barnards Star')
    # fig.text(0.5, 0.00, 'Wavelength $[\AA]$', ha='center')
    # fig.text(0.08, 0.5, 'Normalized Flux', va='center', rotation='vertical')
    plt.subplots_adjust(wspace=0.3)
    if plt_name is not None:
        plt.savefig(os.path.join(out_dir, plt_name),bbox_inches='tight')
    plt.show()

def plot_line_list(axes,model,line_list,lrange,plt_epoch):
    for line in line_list[1].data[(line_list[1].data["Wave"] > lrange.min()) * (line_list[1].data["Wave"] < lrange.max())]:
        print(line["Species"])
        axes[0].axvline(np.log(line["Wave"]) + model[0][0].p[plt_epoch],-5,5,c='k',linestyle='dashed',alpha=0.4)

0.025 stellar radii

$1/0.025$

In [ ]:
# ind_array = np.array(np.where(all_rv_array['RV_err_all'] > 1e3))
# ind_model_array = np.unique(ind_array[0])
# print(ind_model_array)
# for model_index in ind_model_array:
#     def something(rv_inds):
#         if len(rv_inds) == 1:
#             rv_inds = np.append(rv_inds,rv_inds[0]+1)
#         orders = np.unique(all_models[model_index].metadata['orders'])
#         ls_means = np.ceil([np.exp(all_data[model_index].xs[xxx][:]) for xxx in rv_inds]).min(axis=1)
        
#         lrange = ls_means[:,None] + np.arange(-5,5,4)[None,:]
#         print(lrange)
#         plt_name = "barnards_spectra_o{}_rv{}.png".format(orders,rv_inds)
#         make_order_plot(all_data[model_index],all_models[model_index],lrange,rv_inds,cpus[0],plt_name)
#     rv_inds = ind_array[1,ind_array[0,:] == model_index]
    
#     row_len = 4
#     if len(rv_inds) > row_len:
        
#         rounds = len(rv_inds)//row_len
#         for rv_inds_i in range(rounds):
#             something(rv_inds[rv_inds_i*row_len:np.min([(rv_inds_i+1)*row_len,len(rv_inds)-1])])
#             if rv_inds_i  > 3:
#                 break
#     else:
#         something(rv_inds)

In [ ]:
# (datasets,models,size_n,size_m,plt_epochs,orders)
order_list = [np.min(model.metadata['orders']) for model in all_models]
# print(order_list)
ordered_order_list = np.array(np.argsort(order_list))
# print(ordered_order_list)

plt_epochs = np.arange(0,36,1,dtype=int)
make_grid_plots([all_data[xx] for xx in ordered_order_list],[all_models[xx] for xx in ordered_order_list],5,4,plt_epochs,cpus[0],\
                plt_name="peg51_pre_norm_{}-{}.png".format(plt_epochs.min(),plt_epochs.max()))

In [ ]:
make_deriv_plots(all_data[0],all_models[0],0,cpus[0],'info.png')

In [ ]:
model_index = 20
rv_inds = [-2,-1,0,1]
# rv_data = jabble.physics.velocities(all_models[jjj][0][0].p)
print(all_models[model_index].metadata.keys())
orders = np.unique(all_models[model_index].metadata['orders'])

# dataset, _, _, _, _ = get_dataset(file_b,orders,cpus[0])
ls_means = np.ceil([np.exp(all_data[model_index].xs[xxx][:]) for xxx in rv_inds]).mean(axis=1)
lrange = ls_means[:,None] + np.arange(-5,5,2)[None,:]
# print(lrange)
# lrange = np.repeat(lrange,len(rv_inds),axis=0) + np.zeros(len(rv_inds))[:,None]
plt_name = "51peg_spectra_{}.png".format(orders)
print(lrange)
make_order_plot(all_data[model_index],all_models[model_index],lrange,rv_inds,cpus[0],plt_name)
#(dataset,model,lrange,rv_inds,device,plt_name=None)

In [ ]:
from astropy.io import fits

In [ ]:
line_fits = fits.open('~/wobble_jax/data/J_ApJ_795_23_table2.dat.fits')

In [ ]:
def gaussian_smooth(x,sigma):

    return np.exp(-x**2/(2*sigma**2))

def convolve_load(file,sigma_l,xl,eval_a):
    # This reads in line by line a phoenix model
    # storing flux and x values to the cache around the grid point
    # then convolves to instrument resolution
    
    xl_ii   = 0
    xl_curr = xl[xl_ii]
    fl = np.zeros(xl.shape)

    x_cache = np.array([])
    f_cache = np.array([])

    # xh = []
    # fh = []
    with open(file,'r') as file_stream:
        cnt = 0
        for line in file_stream:
            cnt += 1
            if cnt > 8:
                words = line.split('    ')
                for word in words:
                    try:
                        x_val = np.log(np.double(word))
                        break
                    except ValueError:
                        pass
                        
                if x_val > (xl_curr - eval_a):
                    x_cache = np.append(x_cache,x_val)
                    # xh.append(x_val)
                    for word in words[::-1]:  
                        try:
                            f_cache = np.append(f_cache,np.double(word))
                            # fh.append(np.double(word))
                            break
                        except ValueError:
                            pass
                if x_val > (xl_curr + eval_a):
                    kern = gaussian_smooth(x_cache - xl_curr,sigma_l)
                    if xl_ii+1 >= len(xl):
                        break
                    else:
                        fl[xl_ii] = np.dot(f_cache,kern)/np.sum(kern)
                        xl_ii += 1
                        xl_curr = xl[xl_ii]
                        # print(x_cache.shape,f_c
                        f_cache = f_cache[x_cache > xl_curr - eval_a]
                        x_cache = x_cache[x_cache > xl_curr - eval_a]
                        

    return fl
                    
# x_cache = np.concatenate((x_cache,[np.double(word)]))
low_resolution = 115_000
x_min = np.log(4000)
x_max = np.log(4200)
sigma_l = jabble.physics.delta_x(low_resolution)

xl = np.arange(x_min,x_max,sigma_l/2)
fl = convolve_load('../../wobble_jax/data/models_1721076003/bt-settl/lte032-5.0-0.5a+0.2.BT-NextGen.7.dat.txt',\
                       sigma_l,xl,3*sigma_l)

In [ ]:
# plt.plot(xl,fl,label='low',alpha=0.3)
# plt.plot(xl,fl,marker='.',label='low',alpha=0.4)
# plt.plot(xh,fh,label='high',alpha=0.3)
# # plt.plot(xl.mean() + x_space,y_space,label='kernel')
# plt.xlim(np.log(4010),np.log(4020))
# plt.legend()
# plt.show()

In [ ]:
import nifty_ls
import nifty_ls
from astropy.timeseries import LombScargle

In [ ]:
# def nifty_ls_diagram(dates,rvs,error,title,plt_name):
#     # print(dates)
#     nifty_res = nifty_ls.lombscargle(dates, rvs, dy=error, fmin=0.01, fmax=0.3)

#     # help(nifty_res)
#     plt.figure(figsize=(10,4))
#     plt.plot(1/nifty_res.freq(),nifty_res.power,'-m',markersize=0.2,alpha=0.7)
#     plt.xlabel('Period (Days)')
#     plt.ylabel('Power')
#     plt.title(title)
#     plt.ylim(0.0,0.3)
#     plt.savefig(os.path.join(out_dir, plt_name))
#     plt.show()

# # print(np.array(file_b['dates']).shape,np.array(jabble_file['RV']).shape,np.array(jabble_file['RV_err']).shape)
# # print(np.array(file_b['dates']).shape,np.array(jabble_file['RV_comb']).shape,np.array(jabble_file['RV_comb_err']).shape)
# plt_name = 'ls_jabble_noberv.png'
# nifty_ls_diagram(np.array(file_b['dates']),rv_array['RV_comb'] + np.array(file_b['bervs']),rv_array['RV_err_comb'],title="Jabble - BERV",plt_name=plt_name)
# plt_name = 'ls_hires_noberv.png'
# nifty_ls_diagram(np.array(file_b['dates']),np.array(file_b['pipeline_rvs']) + np.array(file_b['bervs']),np.array(file_b['pipeline_sigmas']),title="HIRES - BERV",plt_name=plt_name)
# plt_name = 'ls_berv.png'
# nifty_ls_diagram(np.array(file_b['dates']),np.array(file_b['bervs']),None,title="BERVS",plt_name=plt_name)

In [ ]:
# all_models[-1].results

In [ ]:
frequency, power = LombScargle(np.array(file_b['dates']), jabble_file['RV_comb']).autopower(method="fastnifty")

In [ ]:
def plot_earth_residual_img(model,dataset,lrange,orders,rest_shifts,residual_resolution,plt_name,line_list,device):
    xrange = np.log(lrange)
    xmin, xmax = np.min(xrange), np.max(xrange)
    # xinds = ((dataset[0].xs[:] < xmax) * (dataset[0].xs[:] > xmin)).astype(bool)
    data, meta, keys = dataset.blockify(device,return_keys=True)

    # create residual image of the size of the number of data epochs times the number of orders
    residual_img = np.zeros((np.sum([np.sum(keys["orders"][meta["orders"]] == order) for order in orders]),residual_resolution))
    
    fig, ax = plt.subplots(2,2,figsize=(8, 8),height_ratios=[1,4],width_ratios=[4,1],sharex='col',sharey='row')
    fig.tight_layout()

    max_shift, min_shift = np.max(rest_shifts), np.min(rest_shifts)
    new_grid = np.linspace(xmin,xmax,residual_resolution)

    epsilon = np.log(np.mean(lrange) + 1) - np.log(np.mean(lrange))
    model.fix()
    model.display()

    # print(np.sum(np.log(line_list[1].data["Wave"]) > xmin),np.sum(np.log(line_list[1].data["Wave"]) < xmax),\
    #      np.sum((np.log(line_list[1].data["Wave"]) > xmin)*(np.log(line_list[1].data["Wave"]) < xmax)))
    # for line in line_list[1].data[(np.log(line_list[1].data["Wave"]) > xmin) * (np.log(line_list[1].data["Wave"]) < xmax)]:
    #     print(line["Species"])
    #     ax[1,0].vlines(np.log(line["Wave"]),0,len(dataset))
    # for line in list_list

    plt_epochs = np.concatenate([np.array(np.where(keys["orders"][meta["orders"]] == order)).flatten() for order in orders])
    for i,plt_epoch in enumerate(plt_epochs):
        datarow = jabble.loss.dict_ele(data,plt_epoch,device)
        metarow = jabble.loss.dict_ele(meta,plt_epoch,device)
        
        xless = (dataset[plt_epoch].xs[:] <= (xmax + epsilon))
        xmore = (dataset[plt_epoch].xs[:] >= (xmin - epsilon)) #+ rest_shifts[plt_epoch] 
        xinds = (xless * \
                 xmore).astype(bool)
        # print(np.sum(xmore),np.sum(xless))
        x_grid = dataset[plt_epoch].xs[(~dataset[plt_epoch].mask)*xinds]
        y_grid = dataset[plt_epoch].ys[(~dataset[plt_epoch].mask)*xinds]
        residual = (y_grid - model([],x_grid,metarow))#*jnp.sqrt(dataset[plt_epoch].yivar[(~dataset[plt_epoch].mask)*xinds])

        if np.sum(xless) == 0:
            x_grid = np.array([dataset[plt_epoch].xs[~dataset[plt_epoch].mask].min()])
            residual = np.array([0.0])
        if np.sum(xmore) == 0:
            x_grid = np.array([dataset[plt_epoch].xs[~dataset[plt_epoch].mask].max()])
            residual = np.array([0.0])
        # print(residual.shape,np.sum(dataset[plt_epoch].mask*xinds),np.sum(xinds),np.sum(dataset[plt_epoch].mask))
        residual_img[i,:] = scipy.interpolate.interp1d(x_grid,residual,kind='nearest',bounds_error=False,fill_value=0.0)(new_grid )#+ \rest_shifts[plt_epoch]
    cmap = plt.get_cmap("RdBu")

    ax[0,1].axis('off')
    ax[0,0].step(new_grid,       np.sqrt((residual_img**2).mean(axis=0)),'k',where='mid',zorder=1,alpha=0.3,ms=3)
    # ax[0,0].step(new_grid,       (residual_img).sum(axis=0),'m',where='mid',zorder=1,alpha=0.3,ms=3)
    ax[1,1].step(np.sqrt((residual_img**2).mean(axis=1)),np.arange(len(plt_epochs))[::-1],'k',where='post',zorder=1,alpha=0.3,ms=3)
    
    
    # ax[1,0].set_ylim(0,np.max(orders)+1)
    ax[1,0].set_xlim(xmin,xmax)
    extent = [xmin,xmax,0,len(dataset)+1]
    ax[1,0].imshow(residual_img,cmap=cmap,aspect="auto",vmin=-0.1,vmax=0.1,extent=extent,interpolation='nearest')
    ax[1,0].set_xlabel('Wavelength [$\AA$]')
    ax[1,0].set_ylabel('Chunks')
    # # plt.xticks([])
    ax[1,0].set_xticks(xrange)
    ax[1,0].set_xticklabels(['{:0.1f}'.format(l) for l in lrange])
    # ax[1,0].get_shared_x_axes().join(ax[1,0], ax[1,1])
    # plt.xlabel()
    if plt_name is not None:
        plt.savefig(os.path.join(out_dir, plt_name))
    plt.show()
    plt.show()


    # worst_epochs = np.zeros(len(orders),dtype=bool)
    # worst_epochs = (residual_img**2).sum(axis=1) > 0.7
    # print(worst_epochs)
    # print(orders)
    # print(orders[worst_epochs])
    # return orders[worst_epochs]

In [ ]:
l_min = np.exp(np.array([np.min([np.min(xs) for xs in dataset.xs]) for dataset in all_data]))
l_max = np.exp(np.array([np.max([np.max(xs) for xs in dataset.xs]) for dataset in all_data]))

In [ ]:
ls_midpoint = 5677.000
lrange = np.ceil(ls_midpoint) + np.arange(-5.0,5.0,2.0)
model_index = np.where((ls_midpoint > l_min)*(ls_midpoint < l_max))[0][0]
print(model_index)

# orders = np.unique(all_models[model_index].metadata['orders'])

# data, meta = dataset.blockify(cpus[0])
rest_shifts = all_models[model_index][0][0].p
index_sort = np.argsort(rest_shifts)
residual_resolution = 256
orders_plt = np.unique(all_models[model_index].metadata['orders'])
plt_name = "res_img_barn_o{}-{}_l{}-{}_.png".format(np.min(orders_plt),np.max(orders_plt),np.min(lrange),np.max(lrange))
plot_earth_residual_img(all_models[model_index],all_data[model_index],lrange,orders_plt,\
                        rest_shifts,residual_resolution,plt_name,None,device=cpus[0])

rv_inds = [-1,-2,0,1]
lrange_stack = np.ones(len(rv_inds))[:,None] * lrange[None,:]
plt_name = "spectra_barn_o{}-{}_l{}-{}_.png".format(orders_plt.min(),orders_plt.max(),np.min(lrange),np.max(lrange))
# make_order_plot(all_data[model_index],all_models[model_index],lrange,rv_inds,cpus[0],plt_name)
shift_by_eye = 0#np.log(5001/5000)
make_order_plot(all_data[model_index],all_models[model_index],lrange_stack,rv_inds,cpus[0],plt_name,line_list=line_list)

In [ ]:
# lmin = 5580
# lmax = 5585
# lspacing = 1
# residual_resolution = 2048
# lrange = np.arange(lmin,lmax+lspacing,lspacing)
# plt_name =  "02-{}_res_img_starref.png".format(os.path.split(model_name_b)[-1],np.min(orders),np.max(orders))

# rest_vel = model_b[0][0].p#np.zeros(.shape)
# index_sort = np.argsort(rest_vel)
# rest_shifts =  model_b[0][0].p#jabble.physics.shifts(rest_vel)

In [ ]:
# for filename in dir:
#     model = jabble.model.load(filename)
#     rv_data = jabble.physics.velocities(model[0][0].p)
#     orders = np.unique(model.metadata['orders'])
#     lmean = np.mean(np.exp(model[0][1].xs))
#     lmin = lmean-10
#     lmax = lmean+10
#     lspace = 4
    
#     lrange = np.arange(lmin,lmax+lspace,lspace)
    
#     rv_inds = [-1,0,1]
#     make_order_plot(file_b,model,lmin,lmax,lrange,[orders[1]],rv_inds,cpus[0])

In [ ]:
# def imshow_rv_difference(all_models,times_t,times_e,rv_e,err_e,bervs,):
    
#     rv_array = np.zeros((len(all_models),*all_models[0][0][0].p.shape))
#     order_array = []

#     info_e = 1/err_e**2
#     epoches_span_e = np.zeros(len(rv_e))

#     rv_comb = jnp.array(len(times_t))
#     for iii,time in enumerate(times_t):
#         indices = times_e == time

#         rv_indiv[indices] = rv_e[indices] + bervs[iii] - correct_vel[iii]
#         epoches_span_e[indices] = iii
#         rv_comb = rv_comb.at[iii].set(jnp.dot(info_e[indices],rv_e[indices])/jnp.sum(info_e[indices]))
#         info_comb = info_comb.at[iii].set((jnp.dot(info_e[indices],rv_e[indices]**2)/jnp.sum(info_e[indices])) - rv_comb[iii]**2)


#     for i,model in enumerate(all_models):
#         # data, _, _, full_init_shifts, _ = get_dataset(file_b,orders,cpus[0])
#         datablock, metablock, keys = data.blockify(cpus[0],return_keys=True)
#         rv_array[i,:] = jabble.physics.velocities(model[0][0].p)
#         order_array.append(np.unique(model.metadata['orders']))



In [ ]:
# imshow_rv_difference(all_models)

5577 night sky line

In [ ]:
# modelname = 'barnardsvmapmodel1.mdl'
# # model = jabble.model.load(modelname)
# jabble.model.save(modelname,model)

In [ ]:
# def make_plot(model,dataset,init_shifts,filename):
#     x_window = np.log(4550) - np.log(4549)
#     lmin = np.exp(dataset.xs[0,500])
#     lmax = np.exp(dataset.xs[0,1500])
#     lrange = np.arange(lmin,lmax,5)
#     plt_unit = u.Angstrom
#     epoches = 25
#     r_plots = 5

#     vel_epoch = 5
#     fig, axes = plt.subplots(
#         epoches // r_plots,
#         r_plots,
#         figsize=(8, 8),
#         sharex=False,
#         sharey=True,
#         facecolor=(1, 1, 1),
#         dpi=200,
#     )
#     # fig.suptitle(filenames[model_num])
#     for plt_epoch in range((epoches // r_plots) * r_plots):
#         xplot = np.linspace(np.log(lmin), np.log(lmax), dataset.xs.shape[1] * 10)
#         axes[plt_epoch // r_plots, plt_epoch % r_plots].set_xlim(
#             xplot.min() + model[0][0].p[plt_epoch],
#             xplot.max() + model[0][0].p[plt_epoch],
#         )

#         # model_set[model_num].fix()
#         # model_set[model_num].fit(0)
#         # rv_model_deriv = jax.jacfwd(model_set[model_num], argnums=0)(model_set[model_num].get_parameters(),dataset.xs[plt_epoch,:],plt_epoch)
#         # rv_loss_deriv = jax.jacfwd(loss, argnums=0)(model_set[model_num].get_parameters(),datasets[0],vel_epoch,model_set[model_num])

#         model.fix()

#         axes[plt_epoch // r_plots, plt_epoch % r_plots].errorbar(
#             dataset.xs[plt_epoch, :],
#             dataset.ys[plt_epoch, :],
#             dataset.yerr[plt_epoch, :],
#             fmt=".k",
#             elinewidth=1.2,
#             zorder=1,
#             alpha=0.5,
#             ms=3,
#         )

#         # true_model.fix()

#         axes[plt_epoch // r_plots, plt_epoch % r_plots].plot(
#             xplot,
#             model([], xplot, plt_epoch),
#             "-r",
#             linewidth=1.2,
#             zorder=2,
#             alpha=0.5,
#             ms=6,
#         )
#         # axes[plt_epoch // r_plots, plt_epoch % r_plots].plot(xplot,true_model([],xplot,plt_epoch),'-r',linewidth=1.2,zorder=1,alpha=0.5,ms=6)

#         axes[plt_epoch // r_plots, plt_epoch % r_plots].set_ylim(-2, 1)
#         #         axes[i,j].set_yticks([])
#         axes[plt_epoch // r_plots, plt_epoch % r_plots].set_xticks(np.log(lrange))
#         axes[plt_epoch // r_plots, plt_epoch % r_plots].set_xticklabels(
#             ["{:2.0f}".format(x) for x in lrange]
#         )

#         res_ax = axes[plt_epoch // r_plots, plt_epoch % r_plots].twinx()
#         residual = loss(
#             model.get_parameters(),
#             dataset,
#             plt_epoch,
#             model,
#         )
#         res_ax.step(
#             dataset.xs[plt_epoch, :], residual, where="mid", alpha=0.3, label="residual"
#         )
#         res_ax.set_ylim(0.0, 20)
#         res_ax.set_yticks([])
#         # res_ax.step(model_set[i][j][1].xs+model_set[i][j][0].p[plt_epoch],\
#         #             model_set[i][j].results[-2]['grad'][:],\
#         #             where='mid',alpha=0.4,label='residual',zorder=-1)
#         # res_ax.set_yticks([])

#         # res_ax.step(x_grid,\
#         #             rv_model_deriv[:,plt_epoch],\
#         #             where='mid',alpha=0.4,label='RV Derivative',zorder=-1)

#         #     res_ax.step(x_grid,\
#         #                 rv_loss_deriv[:,plt_epoch],\
#         #                 where='mid',alpha=0.4,label='RV Derivative',zorder=-1)

#         #     align_yaxis(, 0, , 0)

#         align.yaxes(
#             axes[plt_epoch // r_plots, plt_epoch % r_plots], 0.0, res_ax, 0.0, 2.0 / 3.0
#         )

#     # res.get_shared_y_axes().join(ax1, ax3)
#     fig.text(0.5, 0.04, "$\lambda$", ha="center")
#     fig.text(0.04, 0.5, "y", va="center", rotation="vertical")
#     # fig.text(0.96, 0.5, '$d \L /d \delta x$', va='center', rotation=270)
#     # fig.text(0.96, 0.5, '$d f_{{{ji}}} /d \delta x_k$', va='center', rotation=270)
#     fig.text(0.96, 0.5, "residuals", va="center", rotation=270)

#     plt.savefig(
#         os.path.join(out_dir, "02-res_{}.png".format(filename)),
#         dpi=300,
#         bbox_inches="tight",
#     )
#     plt.show()

6563 h alpha

In [ ]:
# filenames = ['51peg','barnards']
# make_plot(model_b,dataset_b,shifts_b,filenames[1])

In [ ]:
# with open('../data/models_1721076003/bt-settl/lte032-5.0-0.5a+0.2.BT-NextGen.7.dat.txt', 'r') as f:
#     # text = f.read()
#     cnt = 0
#     for line in f:
        
#         cnt += 1
#         if cnt > 8:
#             print([np.double(x) for x in line.split("    ")[-2:]])

#         if cnt > 12:
#             break
        

# # print(text)

In [ ]:
# def make_better_plot(model_set,datasets,file_set):
    

#     fig, axes = plt.subplots(2*len(model_set),4,figsize=(4*4,4*len(model_set)),sharex=True,facecolor=(1, 1, 1),dpi=200,height_ratios=[4,1]*len(model_set))
    
#     for jj,(model,dataset,file) in enumerate(zip(model_set,datasets,file_set)):
#         x_window = np.log(4550) - np.log(4549)
#         lmin = np.exp(dataset.xs[0,0])
#         lmax = np.exp(dataset.xs[0,2000])
#         lrange = np.arange(lmin,lmax,5)
#         sort_airmasses = np.argsort(np.array(file['airms'][:]))
#         plt_epochs = np.concatenate((sort_airmasses[:2],sort_airmasses[-2:]))
        
        
        
        
#         offset = 1.0
#         xplot = np.linspace(np.log(lmin)-x_window,np.log(lmax)+x_window,dataset.xs.shape[1]*10)
#         for ii,plt_epoch in enumerate(plt_epochs):
#             axes[2*jj,ii].set_xlim(xplot.min()+model[0][0].p[plt_epoch],xplot.max()+model[0][0].p[plt_epoch])
            
#             model.fix()
            
#             axes[2*jj,ii].errorbar(dataset.xs[plt_epoch,:],dataset.ys[plt_epoch,:],\
#                                      dataset.yerr[plt_epoch,:],fmt='.k',elinewidth=1.2,zorder=1,alpha=0.5,ms=3)
            
#             axes[2*jj,ii].plot(xplot,offset + model[0]([],xplot,plt_epoch),'-r',linewidth=1.2,zorder=2,alpha=0.7,ms=6)
#             axes[2*jj,ii].plot(xplot,2*offset + model[1]([],xplot,plt_epoch),'-b',linewidth=1.2,zorder=2,alpha=0.7,ms=6)
#             axes[2*jj,ii].plot(xplot,model[2]([],xplot,plt_epoch),'-m',linewidth=1.2,zorder=3,alpha=0.7,ms=6)
            
#             # axes[0,ii].plot(xplot,2*offset + model[1]([],xplot,plt_epoch),'-b',linewidth=1.2,zorder=2,alpha=0.7,ms=6)
#             # axes[0,ii].plot(xplot,offset + model([],xplot,plt_epoch),'-g',linewidth=1.2,zorder=2,alpha=0.7,ms=6)
#             # axes[plt_epoch // r_plots, plt_epoch % r_plots].plot(xplot,true_model([],xplot,plt_epoch),'-r',linewidth=1.2,zorder=1,alpha=0.5,ms=6)
            
            
#             axes[2*jj,ii].set_ylim(-2,3)
#             axes[2*jj,ii].set_xticks([])
#             # axes[0].set_yticks([])
#             axes[2*jj+1,ii].set_xticks(np.log(lrange))
#             axes[2*jj+1,ii].set_xticklabels(['{:2.0f}'.format(x) for x in lrange])
            
#             axes[2*jj+1,ii].plot(dataset.xs[plt_epoch,:],dataset.ys[plt_epoch,:] - model([],dataset.xs[plt_epoch,:],plt_epoch),'.k',alpha=0.4,ms=1)
            
#             axes[2*jj+1,ii].set_ylim(-0.1,0.1)
#             axes[2*jj,ii].set_title('airmass = {}'.format(file['airms'][:][plt_epoch]))
#         # res_ax = axes[plt_epoch // r_plots, plt_epoch % r_plots].twinx()
#         # residual = loss(model_set[model_num].get_parameters(),dataset,plt_epoch,model_set[model_num])
#         # res_ax.step(dataset.xs[plt_epoch,:],residual,where='mid',alpha=0.3,label='residual')
#         # res_ax.set_ylim(0.0,20)
#         # res_ax.set_yticks([])
#         # res_ax.step(model_set[i][j][1].xs+model_set[i][j][0].p[plt_epoch],\
#         #             model_set[i][j].results[-2]['grad'][:],\
#         #             where='mid',alpha=0.4,label='residual',zorder=-1)
#         # res_ax.set_yticks([])
        
#         # res_ax.step(x_grid,\
#         #             rv_model_deriv[:,plt_epoch],\
#         #             where='mid',alpha=0.4,label='RV Derivative',zorder=-1)
            
#         #     res_ax.step(x_grid,\
#         #                 rv_loss_deriv[:,plt_epoch],\
#         #                 where='mid',alpha=0.4,label='RV Derivative',zorder=-1)
            
#         #     align_yaxis(, 0, , 0)
            
#             # align.yaxes(axes[plt_epoch // r_plots, plt_epoch % r_plots], 0.0, res_ax, 0.0, 2./3.)
        
#         # res.get_shared_y_axes().join(ax1, ax3)
#         fig.text(0.5, 0.04, '$\lambda$', ha='center')
#         # fig.text(0.04, 0.5, 'y', va='center', rotation='vertical')
#         # fig.text(0.96, 0.5, '$d \L /d \delta x$', va='center', rotation=270)
#         # fig.text(0.96, 0.5, '$d f_{{{ji}}} /d \delta x_k$', va='center', rotation=270)
#         # fig.text(0.96, 0.5, 'residuals', va='center', rotation=270)
    
#     # plt.savefig(os.path.join(out_dir,'02-full-barn-51peg.png'),dpi=300,bbox_inches='tight')
#     plt.show()

In [ ]:
# make_better_plot(model_set,datasets,file_set)

In [ ]:
# tell_loss = [[],[]]
# for jjj, (dataset, model) in enumerate(zip(datasets,model_set)):
#     for iii in range(dataset.ys.shape[0]):
#         tell_loss[jjj].append(loss([],dataset,iii,model[0]).sum())

# plt.plot(np.array(file_p['airms'][:]),tell_loss[0],'.k',label='51 peg')
# plt.plot(np.array(file_b['airms'][:]),tell_loss[1],'.r',label='barnards')
# # plt.ylim(0.0,5e4)

# # plt.plot(np.array(file_p['airms'][:]),model_p[1][1].p,'.k',label='51 peg')
# # plt.plot(np.array(file_b['airms'][:]),model_b[1][1].p,'.r',label='barnards')
# plt.xlabel('airmass')
# plt.ylabel('$\Sigma_* (y_* - \hat{y}_s(x_*)) I_{y*}$')
# # plt.plot()
# plt.legend()
# # plt.savefig(os.path.join(out_dir,'02-airmass_loss.png'),dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
# plt.plot(np.array(file_p['airms'][:]),model_p[1][1].p,'.k',label='51 peg')
# plt.plot(np.array(file_b['airms'][:]),model_b[1][1].p,'.r',label='barnards')
# plt.xlabel('airmass')
# plt.ylabel('~a')
# x_space = np.linspace(np.min(np.array(file_b['airms'][:])),np.max(np.array(file_b['airms'][:])))
# plt.plot(x_space,x_space,'-.k',alpha=0.3)
# plt.legend()
# # plt.savefig(os.path.join(out_dir,'02-airmass_an.png'),dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
# fig, axes = plt.subplots(1,figsize=(4,4),sharex=True,facecolor=(1, 1, 1),dpi=200)

# plt_epoch = 10
# x_window = np.log(4550) - np.log(4549)
# lmin = np.exp(dataset_p.xs[0,500])
# lmax = np.exp(dataset_p.xs[0,1500])
# lrange = np.arange(lmin,lmax,5)
# xplot = np.linspace(np.log(lmin)-x_window,np.log(lmax)+x_window,dataset_p.xs.shape[1]*10)
# axes.plot(xplot,model_p[1]([],xplot,plt_epoch),'-b',linewidth=1.2,zorder=2,alpha=0.6,ms=6,label='51 peg')
# axes.plot(xplot,0.05 + model_b[1]([],xplot,plt_epoch),'-r',linewidth=1.2,zorder=2,alpha=0.6,ms=6,label='barnard')
# axes.legend()

# axes.set_ylim(-0.2,0.1)
# axes.set_xticks([])
# axes.set_ylabel('log flux + offset')
# axes.set_xlabel('$\lambda$')
# axes.set_xticks(np.log(lrange))
# axes.set_xticklabels(['{:2.0f}'.format(x) for x in lrange])
# plt.title('just tellurics')
# # plt.savefig(os.path.join(out_dir,'02-airmass-tell.png'),dpi=300,bbox_inches='tight')
# plt.show()